In [ ]:
import pandas as pd
from alpha_vantage.fundamentaldata import FundamentalData
import time
import os # For securely getting the API key

api_key = ''

symbol = 'ASML'

try:
    fd = FundamentalData(key=api_key, output_format='pandas')
except ValueError as e:
    print(f"Error initializing Alpha Vantage client: {e}")
    print("Please ensure your API key is valid.")
    exit() # Exit if initialization fails

data_frames = {
    "income_annual": None,
    "income_quarterly": None,
    "balance_annual": None,
    "balance_quarterly": None,
    "cashflow_annual": None,
    "cashflow_quarterly": None,
}
fetch_status = {}

# Helper function to fetch data and handle errors/rate limits
def fetch_alpha_vantage_data(func, sym, report_name):
    """Fetches data using the provided function, handles common errors."""
    print(f"Attempting to fetch {report_name} data for {sym}...")
    try:
        data, meta_data = func(symbol=sym)
        data = data.T
        for col in data.columns:
            if data[col].astype(str).str.replace('.', '', 1).str.isdigit().all():
                 data[col] = pd.to_numeric(data[col], errors='coerce')

        print(f"Successfully fetched {report_name} data.")
        return data, True
    except ValueError as e:
        # Common error for invalid symbol or no data available
        print(f"Error fetching {report_name}: {e}. Symbol might be invalid or data unavailable.")
        return None, False
    except Exception as e:
        # Catch other potential errors (API limits, network issues)
        print(f"An unexpected error occurred fetching {report_name}: {e}")
        # You might implement retries or specific handling here if needed
        return None, False

# Income Statement - Annual
df, status = fetch_alpha_vantage_data(fd.get_income_statement_annual, symbol, "Annual Income Statement")
data_frames["income_annual"] = df
fetch_status["income_annual"] = status
time.sleep(15) # Add delay to respect free tier API limits (e.g., 5 calls/min)

# Income Statement - Quarterly
df, status = fetch_alpha_vantage_data(fd.get_income_statement_quarterly, symbol, "Quarterly Income Statement")
data_frames["income_quarterly"] = df
fetch_status["income_quarterly"] = status
time.sleep(15)

# Balance Sheet - Annual
df, status = fetch_alpha_vantage_data(fd.get_balance_sheet_annual, symbol, "Annual Balance Sheet")
data_frames["balance_annual"] = df
fetch_status["balance_annual"] = status
time.sleep(15)

# Balance Sheet - Quarterly
df, status = fetch_alpha_vantage_data(fd.get_balance_sheet_quarterly, symbol, "Quarterly Balance Sheet")
data_frames["balance_quarterly"] = df
fetch_status["balance_quarterly"] = status
time.sleep(15)

# Cash Flow - Annual
df, status = fetch_alpha_vantage_data(fd.get_cash_flow_annual, symbol, "Annual Cash Flow")
data_frames["cashflow_annual"] = df
fetch_status["cashflow_annual"] = status
time.sleep(15)

# Cash Flow - Quarterly
df, status = fetch_alpha_vantage_data(fd.get_cash_flow_quarterly, symbol, "Quarterly Cash Flow")
data_frames["cashflow_quarterly"] = df
fetch_status["cashflow_quarterly"] = status

all_successful = True
for name, status in fetch_status.items():
    print(f"{name}: {'Success' if status else 'Failed'}")
    if not status:
        all_successful = False

if not all_successful:
    print("\nWarning: Some data could not be fetched. Check error messages above.")

# You can assign them to individual variables if you prefer:
income_statement_annual_df = data_frames["income_annual"]
income_statement_quarterly_df = data_frames["income_quarterly"]
balance_sheet_annual_df = data_frames["balance_annual"]
balance_sheet_quarterly_df = data_frames["balance_quarterly"]
cash_flow_annual_df = data_frames["cashflow_annual"]
cash_flow_quarterly_df = data_frames["cashflow_quarterly"]

excel_filename = f"{symbol}_financial_statements.xlsx"

# Check if any data was successfully fetched before attempting to write
if any(df is not None for df in data_frames.values()):
    try:
        # Use pandas ExcelWriter to save multiple sheets in one file
        with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
            print(f"Creating Excel file: {excel_filename}")

            # Loop through the dictionary of DataFrames
            for sheet_name, df in data_frames.items():
                # Only write the sheet if the DataFrame was successfully fetched
                if df is not None:
                    print(f"Writing sheet: {sheet_name}...")
                    df.to_excel(writer, sheet_name=sheet_name, index=True)
                else:
                    print(f"Skipping sheet: {sheet_name} (data not fetched).")

        print(f"\nSuccessfully saved all available data to '{excel_filename}'")
        # Reminder about the current date based on the context provided
        print(f"(File saved on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')})")


    except Exception as e:
        print(f"\nError saving data to Excel file '{excel_filename}': {e}")
        print("Please ensure you have 'openpyxl' installed (`pip install openpyxl`)")
        print("Also check file permissions and if the file is already open elsewhere.")

else:
    print("\nNo data was successfully fetched. Excel file not created.")

print("\nScript finished.")

Attempting to fetch Annual Income Statement data for ASML...
Successfully fetched Annual Income Statement data.
Attempting to fetch Quarterly Income Statement data for ASML...
Successfully fetched Quarterly Income Statement data.
Attempting to fetch Annual Balance Sheet data for ASML...
Successfully fetched Annual Balance Sheet data.
Attempting to fetch Quarterly Balance Sheet data for ASML...
Successfully fetched Quarterly Balance Sheet data.
Attempting to fetch Annual Cash Flow data for ASML...
Successfully fetched Annual Cash Flow data.
Attempting to fetch Quarterly Cash Flow data for ASML...
Successfully fetched Quarterly Cash Flow data.
income_annual: Success
income_quarterly: Success
balance_annual: Success
balance_quarterly: Success
cashflow_annual: Success
cashflow_quarterly: Success
Creating Excel file: ASML_financial_statements.xlsx
Writing sheet: income_annual...
Writing sheet: income_quarterly...
Writing sheet: balance_annual...
Writing sheet: balance_quarterly...
Writing s

In [3]:
income_statement_annual_df

date,1970-01-01 00:00:00.000000000,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000002,1970-01-01 00:00:00.000000003,1970-01-01 00:00:00.000000004,1970-01-01 00:00:00.000000005,1970-01-01 00:00:00.000000006,1970-01-01 00:00:00.000000007,1970-01-01 00:00:00.000000008,1970-01-01 00:00:00.000000009,1970-01-01 00:00:00.000000010,1970-01-01 00:00:00.000000011,1970-01-01 00:00:00.000000012,1970-01-01 00:00:00.000000013,1970-01-01 00:00:00.000000014,1970-01-01 00:00:00.000000015,1970-01-01 00:00:00.000000016,1970-01-01 00:00:00.000000017,1970-01-01 00:00:00.000000018,1970-01-01 00:00:00.000000019
fiscalDateEnding,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,2010-12-31,2009-12-31,2008-12-31,2007-12-31,2006-12-31,2005-12-31
reportedCurrency,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR,EUR
grossProfit,14492000000,14136100000,10512700000,9809000000,6797200000,4900100000,4718300000,4076700000,3044500000,2895700000,2497370000,2085980000,2005200000,2351742000,1955200000,458392000,1017922261,1562802806,1462016651,973454813
totalRevenue,28262900000,27558500000,21173400000,18611000000,13978500000,11820000000,10944000000,9052800000,6794800000,6287400000,5856277000,5245326000,4731500000,5651035000,4507900000,1596063000,2960682847,3814681368,3597101452,2527046847
costOfRevenue,13770900000,13422400000,10660700000,8802000000,7181300000,6919900000,6225700000,4976100000,3750300000,3391700000,3358907000,3159346000,2726300000,3299293000,2552700000,1137671000,1942760586,2251878562,2135084801,1553592034
costofGoodsAndServicesSold,13770900000,13422400000,10660700000,8802000000,7181300000,6919900000,6225700000,4976100000,3750300000,3391700000,3358907000,3159346000,2726300000,3299293000,2552700000,1137671000,1942760586,2251878562,2135084801,1553592034
operatingIncome,9022600000,9042300000,7321000000,6750100000,4051500000,2716700000,2883300000,2496200000,1657700000,1565100000,1523757000,1275136000,1156800000,1643503000,1250700000,-165013000,287726058,826687341,870651729,448776094
sellingGeneralAndAdministrative,1165700000,1113200000,945900000,725600000,544900000,520500000,488000000,416000000,374760000,345732000,321110000,311741000,259301000,217904000,181045000,156644000,212341000,225668000,204799000,621797000
researchAndDevelopment,4303700000,3980600000,2282100000,2547000000,2200800000,1662900000,1347000000,1259700000,1105800000,1068100000,735947000,563965000,589100000,492184000,523400000,466761000,539600486,486907343,386566618,323628244
operatingExpenses,5469400000,5093800000,3191700000,3272600000,2745700000,2183400000,1835000000,1676300000,1480600000,1413800000,1054619000,875300000,848400000,708239000,704500000,623405000,730196201,736115464,591364921,524679563
